In [709]:
#import pandas as pd
import numpy as np
import time
import gzip

### Constants

In [816]:
RESULT_PATH = "tweet_tokens/text_tokens_unique.csv"
N_ROWS = 100

### Read tweet_text_tokens and remove duplicates and pad all the tweets to match with length = max_length and save them

In [817]:
TWEET_ID = "tweet_features_tweet_id"
TWEET_TOKENS = "tweet_features_text_tokens"

TWEET_ID_FILE = "../mapped_columns/direct_mapping/tweet_features_tweet_id.csv.gz"
TWEET_TOKENS_FILE = "../raw_columns/tweet_features_text_tokens.csv.gz"

#MAX_TWEET_ID = 73549798

tweet_id_dictionary = {}

In [818]:
result_file = open(RESULT_PATH, "w+")

In [819]:
def save(identifier, tokens):
    result_file.write(str(identifier) + ',' + tokens + '\n')

### Write the header (columns names)

In [820]:
save(TWEET_ID, TWEET_TOKENS)

### Open files to be read and save tweet tokens by skipping duplicates

In [821]:
id_file = gzip.open(TWEET_ID_FILE, "rb")
tokens_file = gzip.open(TWEET_TOKENS_FILE, "rb")

In [822]:
%%time

# ~ 20 seconds for 1 million rows
# ~ 40 minutes for the whole training dataset

# ignore header
id_file.readline()
tokens_file.readline()

finished = False
i = 0

while not finished and i < N_ROWS:

    #print(i)

    id_line = str(id_file.readline())
    
    if id_line != '':
        id_line = id_line.replace('b','').replace("'",'').split(',')

        tokens_line = str(tokens_file.readline()).replace('b','').replace("'",'').split(',')
        
        tokens_line[1] = tokens_line[1].replace("\\t", "\t").replace("\\n",'')  # tokens_line[1] is the tokens string

        identifier = id_line[1].replace("\\n", '')  # id_line[1] contains the tweet id

        if identifier not in tweet_id_dictionary:
            tweet_id_dictionary[identifier] = 1
            tokens = tokens_line[1]
            save(identifier, tokens)

        if identifier == MAX_TWEET_ID:
            finished = True
    else:
        finished = True

    i += 1

CPU times: user 20.6 s, sys: 700 ms, total: 21.3 s
Wall time: 21.3 s


In [808]:
id_file.close()
tokens_file.close()

result_file.close()

### Check the dataset

In [629]:
len(tweet_id_dictionary)

73549799

In [633]:
%%time

df = pd.read_csv(RESULT_PATH,
                    names=[TWEET_ID],
                    #compression='gzip',
                    #nrows=N_ROWS,
                    header=0,
                    index_col=0)

CPU times: user 4min 36s, sys: 21.5 s, total: 4min 57s
Wall time: 4min 57s


In [635]:
df

,tweet_features_tweet_id
0,101\t6417\t3410\t3398\t3184\t1909\t56910\t1683...
1,101\t14120\t131\t120\t120\t188\t119\t11170\t12...
2,101\t62342\t10858\t54439\t19571\t22480\t7831\t...
3,101\t58955\t10898\t103305\t1901\t16181\t7168\t...
4,101\t2435\t5656\t2594\t8279\t8623\t1925\t64126...
...,...
73549794,101\t1963\t60907\t12236\t3653\t3116\t1943\t827...
73549795,101\t1910\t11588\t37311\t89744\t111791\t15221\...
73549796,101\t56898\t137\t10950\t39736\t71793\t11211\t1...
73549797,101\t56898\t137\t10474\t11274\t15709\t10253\t2...


In [470]:
df = pd.read_csv(TWEET_ID_FILE,
                    names=[TWEET_ID],
                    compression='gzip',
                    #nrows=N_ROWS,
                    header=0,
                    index_col=0)

In [471]:
len(df[TWEET_ID].unique())

904761